<a href="https://colab.research.google.com/github/arbaz52/patent-generator-RNN/blob/master/Text_Generator_using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connecting to Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Loading the Dataset - OLD - Do not use.

In [1]:
drivepath = "drive/My Drive/rnn/"

#loading the dataset from csv
import pandas as pd

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer

df = pd.read_csv(drivepath+"a95f913b.csv", sep=",")


df = df[['patent_title', 'patent_abstract']]

def get_essay(df):
  return df['patent_title'].fillna("") + ' ' + df['patent_abstract']


tokenizer = Tokenizer(filters='"#$%&*+/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(get_essay(df))

processed_df = df["patent_abstract"].copy()
processed_df["patent_abstract"] = tokenizer.texts_to_sequences(get_essay(df))
print(processed_df.head())

import gc
gc.enable()
del df
gc.collect()

Using TensorFlow backend.


0    " A ""Barometer"" Neuron enhances stability in...
1    " This invention is a novel high-speed neural ...
2    An optical information processor for use as a ...
3    A method and system for intelligent control of...
4    A method and system for intelligent control of...
Name: patent_abstract, dtype: object


14

# Loading and pre-processing

In [1]:

import re


def format_patent(patent):
    """Add spaces around punctuation and remove references to images/citations."""

    # Add spaces around punctuation
    patent = re.sub(r'(?<=[^\s0-9])(?=[.,;?])', r' ', patent)

    # Remove references to figures
    patent = re.sub(r'\((\d+)\)', r'', patent)

    # Remove double spaces
    patent = re.sub(r'\s\s', ' ', patent)
    return patent


drivepath = "drive/My Drive/rnn/"

#loading the dataset from csv
import pandas as pd

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer

df = pd.read_csv(drivepath+"a95f913b.csv", sep=",", nrows = 1500)

abstracts = list(df['patent_abstract'])

tokenizer = Tokenizer(filters='"#$%&*+/:;<=>?@[\\]^_`{|}~\t\n')

formatted = []
for abstract in abstracts:
  formatted.append(format_patent(abstract))
  
tokenizer.fit_on_texts(formatted)
sequences = tokenizer.texts_to_sequences(formatted)

#to see whats in it
" ".join(tokenizer.index_word[n] for n in sequences[0])


import gc
gc.enable()
del formatted, abstracts, df
gc.collect()

Using TensorFlow backend.


7

In [2]:
import numpy as np
features = []
labels = []
feature_length = 50
for seq in sequences:
  for i in range(feature_length, len(seq)):
    extracted = seq[i - feature_length: i + 1]
    features.append(extracted[:-1])
    labels.append(extracted[-1])
    
features = np.array(features)
labels = np.array(labels)
print(features.shape, labels.shape)


import gc
gc.enable()
del sequences
gc.collect()

(137599, 50) (137599,)


0

In [3]:
idx_word = tokenizer.index_word
num_words = len(idx_word) + 1

print("Number of unique words: ", num_words)

Number of unique words:  9348


# One word encoding the labels

In [4]:
import numpy as np
one_hot_labels = np.zeros((len(labels), num_words))
print(one_hot_labels.shape)
for i, idx in enumerate(labels):
  one_hot_labels[i][idx] = 1
  
  
print(one_hot_labels.shape)

import gc
gc.enable()
del labels
gc.collect()

(137599, 9348)
(137599, 9348)


22

# Loading the pretrained embeddings

In [5]:

import numpy as np
glove=np.loadtxt(drivepath+'glove.6B.100d.txt', dtype='str', comments=None)
words, vectors=glove[:,0],glove[:,1:].astype('float32')
word_lookup={word:vector for word, vector in zip(words, vectors)}
print(word_lookup['this'])

import gc
gc.enable()
del glove, words
gc.collect()


[-0.57058   0.44183   0.70102  -0.41713  -0.34058   0.02339  -0.071537
  0.48177  -0.013121  0.16834  -0.13389   0.040626  0.15827  -0.44342
 -0.019403 -0.009661 -0.046284  0.093228 -0.27331   0.2285    0.33089
 -0.36474   0.078741  0.3585    0.44757  -0.2299    0.18077  -0.6265
  0.053852 -0.29154  -0.4256    0.62903   0.14393  -0.046004 -0.21007
  0.48879  -0.057698  0.37431  -0.030075 -0.34494  -0.29702   0.15095
  0.28248  -0.16578   0.076131 -0.093016  0.79365  -0.60489  -0.18874
 -1.0173    0.31962  -0.16344   0.54177   1.1725   -0.47875  -3.3842
 -0.081301 -0.3528    1.8372    0.44516  -0.52666   0.99786  -0.32178
  0.033462  1.1783   -0.072905  0.39737   0.26166   0.33111  -0.35629
 -0.16558  -0.44382  -0.14183  -0.37976   0.28994  -0.029114 -0.35169
 -0.27694  -1.344     0.19555   0.16887   0.040237 -0.80212   0.23366
 -1.3837   -0.023132  0.085395 -0.74051  -0.073934 -0.58838  -0.085735
 -0.10525  -0.51571   0.15038  -0.16694  -0.16372  -0.22702  -0.66102
  0.47197   0.37253 

0

In [6]:
embedding_matrix = np.zeros((num_words, vectors.shape[1]))

for k in idx_word.keys():
  vector = word_lookup.get(idx_word[k], None)
  if vector is not None:
    embedding_matrix[k, :] = vector
    
import gc
gc.enable()
del vectors
gc.collect()

41

In [7]:
print(embedding_matrix.shape, 
      one_hot_labels.shape)

(9348, 100) (137599, 9348)


# Creating the Model

In [32]:
from keras.models import Sequential
from keras.layers import LSTM, CuDNNLSTM, Dense, Dropout, Masking, Embedding

training_length = feature_length

model = Sequential()
model.add(Embedding(input_dim = num_words,
                   input_length = training_length,
                   output_dim = 100,
                   weights = [embedding_matrix],
                   trainable = True,
#                   mask_zero = True
                   ))

#model.add(Masking(mask_value = 0.0))

model.add(LSTM(128,
                    return_sequences = False,
                    dropout = 0.1, recurrent_dropout = 0.1))

model.add(Dropout(0.5))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(num_words, activation = 'softmax'))


model.compile(optimizer = 'adadelta',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 100)           934800    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 9348)              1205892   
Total params: 2,274,452
Trainable params: 2,274,452
Non-trainable params: 0
_________________________________________________________________


# Training the Model

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, one_hot_labels)

print(x_train.shape, y_train.shape,
     x_test.shape, y_test.shape)

(103199, 50) (103199, 9348) (34400, 50) (34400, 9348)


In [0]:

from keras.callbacks import EarlyStopping, ModelCheckpoint

# Create callbacks
callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint(drivepath+'rnn_model.h5', save_best_only=True, 
                             save_weights_only=False)]

In [34]:
hist = model.fit(x_train, y_train,
                  batch_size = 2048, epochs = 150, callbacks=callbacks,
                 shuffle = True,
                  validation_data = (x_test, y_test))

Train on 103199 samples, validate on 34400 samples
Epoch 1/150
103199/103199 [==============================] - 17s 169us/step - loss: 7.2156 - acc: 0.0624 - val_loss: 6.3598 - val_acc: 0.0312
Epoch 2/150
103199/103199 [==============================] - 16s 158us/step - loss: 6.3407 - acc: 0.0744 - val_loss: 6.2670 - val_acc: 0.0312
Epoch 3/150
103199/103199 [==============================] - 16s 156us/step - loss: 6.2640 - acc: 0.0778 - val_loss: 6.1693 - val_acc: 0.0862
Epoch 4/150
103199/103199 [==============================] - 16s 158us/step - loss: 6.2051 - acc: 0.0883 - val_loss: 6.1027 - val_acc: 0.1013
Epoch 5/150
103199/103199 [==============================] - 16s 157us/step - loss: 6.1344 - acc: 0.1007 - val_loss: 6.0344 - val_acc: 0.1015
Epoch 6/150
103199/103199 [==============================] - 16s 151us/step - loss: 6.0821 - acc: 0.1066 - val_loss: 5.9873 - val_acc: 0.1170
Epoch 7/150
103199/103199 [==============================] - 16s 157us/step - loss: 6.0387 - acc:

In [0]:
model.save(drivepath+"rnn-text-generator-added-additional-dense-150-epochs")

In [0]:
model.load_weights(drivepath+"rnn-text-generator-150-epochs")

# Testing the trained Model

In [37]:
"""
print(x_train[122][:25].shape,  x_train[154][:25].shape)
print()
paragraph = " ".join(idx_word[k] for k in (np.concatenate([ x_train[123][:25], x_train[152][:25]])))
"""
paragraph = "The thing with the people is that they do not know what to do or what to say at the right moment , we have been working on this new technology that would help the humans to generate new technologies in a matter of minutes , but unfortunately we do not have enough"
print(paragraph)
n_words_to_generate = 50
for i in range(n_words_to_generate):
  print("working on: " + str(tokenizer.texts_to_sequences([paragraph])[0][-50:]))
  seq = tokenizer.texts_to_sequences([paragraph])[0][-50:]
  seq = np.array([seq])
  next_prob = model.predict(seq)[0]
  ind = np.argmax(next_prob)
  word = idx_word[ind]
  paragraph += " " + word
  
print(paragraph)


The thing with the people is that they do not know what to do or what to say at the right moment , we have been working on this new technology that would help the humans to generate new technologies in a matter of minutes , but unfortunately we do not have enough
working on: [1, 19, 1, 8576, 10, 28, 1041, 1553, 222, 3575, 7, 1553, 18, 3575, 7, 6360, 33, 1, 1187, 3322, 5, 2972, 209, 249, 2611, 26, 81, 232, 807, 28, 1258, 3147, 1, 5587, 7, 106, 232, 1711, 12, 2, 1911, 3, 6557, 5, 599, 2972, 1553, 222, 209, 3478]
working on: [19, 1, 8576, 10, 28, 1041, 1553, 222, 3575, 7, 1553, 18, 3575, 7, 6360, 33, 1, 1187, 3322, 5, 2972, 209, 249, 2611, 26, 81, 232, 807, 28, 1258, 3147, 1, 5587, 7, 106, 232, 1711, 12, 2, 1911, 3, 6557, 5, 599, 2972, 1553, 222, 209, 3478, 3]
working on: [1, 8576, 10, 28, 1041, 1553, 222, 3575, 7, 1553, 18, 3575, 7, 6360, 33, 1, 1187, 3322, 5, 2972, 209, 249, 2611, 26, 81, 232, 807, 28, 1258, 3147, 1, 5587, 7, 106, 232, 1711, 12, 2, 1911, 3, 6557, 5, 599, 2972, 1553, 222